In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astroML.datasets import fetch_sdss_sspp
from astropy.coordinates import SkyCoord
import astropy.units as u

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme(style='white') # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#465BBB', '#3F8FCE', '#7ABBCE', '#3A3865', '#A82F43', '#612A37', '#DC5433', '#F29457']
b1 = '#465BBB'
b2 = '#3F8FCE'
b3 = '#7ABBCE'
b4 = '#3A3865'
black1 = '#22212A'
black2 = '#2D1E21'
or1 = '#A82F43'
or2 = '#612A37'
or3 = '#DC5433'
or4 = '#F29457'
muted_colors = sns.set_palette(sns.color_palette(colors))

# configuring fonts for plotting
font = FontProperties()
font.set_family('serif')
font.set_name('Andale Mono')
font.set_style('normal')
#font.set_size('normal')


%matplotlib inline 

In [3]:
plt.rcParams.update({
    'font.serif': 'Andale Mono',
    'font.family': 'serif'
})

In [4]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import astropy.units as u

In [5]:
Bstars = pd.read_csv('../outputs/FINAL_BE_CATALOGUE.csv')

In [12]:
Bstars.head()

,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,L_chi2dof,L_chi2dofR,Lmean_chi2dof,Lmean_chi2dofR,...,BlazhkoPeakZ,BlazhkoPeriodZ,BpowerRatioZ,BsignificanceZ,Ampl_diff,dP,IndicatorType,ChiType,BE_score,period_vs_amp
0,523832,0.372376,0.372384,0.372380,1.000021,251,1.8,1.2,1.9,1.3,...,2.7122,37.3413,0.1528,6.3617,0.17,0.00002,Z,NaN,NaN,NaN
1,1240665,0.632528,0.632522,0.632525,0.999991,468,25.2,3.0,25.4,2.9,...,1.5865,182.3154,0.0570,15.9957,0.00,0.00001,Z,NaN,NaN,NaN
2,1736308,0.555848,0.555843,0.555846,0.999991,372,2.9,1.3,3.0,1.2,...,1.8264,36.6367,0.0529,5.6501,0.11,0.00001,Z,NaN,NaN,NaN
3,2669011,0.591153,0.591151,0.591152,0.999997,450,2.4,1.1,2.3,1.1,...,1.7015,101.2146,0.0751,6.5360,0.16,0.00000,Z,NaN,NaN,NaN
4,2742032,0.629676,0.629692,0.629684,1.000025,514,1.8,0.9,1.9,1.0,...,1.5956,133.2445,0.0602,7.4751,0.34,0.00003,Z,NaN,NaN,NaN


In [9]:
dataLINEAR = fetch_LINEAR_sample(data_home='../inputs') # fetching the data from astroML data library

In [11]:
dataLINEAR.targets

rec.array([(10003298, 207.52946, 33.70615, 207.529404, 33.706001, 15.73, 1.84,  0.81,  0.31,  0.16,  0.63, 15.89, 0.16, 0.19, 1.12, -0.61, -0.01, -1, 0),
           (10004892, 206.20259, 34.05875, 206.202469, 34.058662, 16.76, 1.14,  0.24,  0.08,  0.01, -0.01, 16.84, 0.24, 0.22, 0.93, -0.17,  0.13, -1, 0),
           (10013411, 206.09627, 36.64865, 206.096115, 36.648674, 15.44, 1.71,  0.75,  0.31,  0.16,  0.62, 15.5 , 0.19, 0.1 , 1.33, -0.45,  0.26, -1, 0),
           ...,
           ( 9984569, 207.759  , -1.79451, 207.758911, -1.794518, 16.49, 1.15, -0.05, -0.04, -0.04,  0.16, 16.89, 0.19, 0.22, 0.74,  0.14, -0.08,  1, 0),
           ( 9987252, 207.2308 , -1.61549, 207.230789, -1.615576, 16.33, 1.17,  0.11, -0.  ,  0.01,  0.52, 16.62, 0.23, 0.28, 1.36, -0.26, -0.15,  1, 0),
           (  999528, 169.0444 ,  5.63446, 169.044342,  5.634354, 16.17, 1.19,  0.12,  0.02, -0.03,  0.63, 16.59, 0.2 , 0.24, 1.05,  0.12, -0.11, -1, 0)],
          dtype=[('objectID', '<i8'), ('raLIN', '<f8'), ('d

In [10]:
dataLINEAR.targets[0]

(10003298, 207.52946, 33.70615, 207.529404, 33.706001, 15.73, 1.84, 0.81, 0.31, 0.16, 0.63, 15.89, 0.16, 0.19, 1.12, -0.61, -0.01, -1, 0)

In [26]:
ra = []
dec = []
Bids = list(Bstars['LINEAR id'])

for i in Bids:
    for j in dataLINEAR.targets:
        if i==j[0]:
            ra.append(j[3])
            dec.append(j[4])

In [28]:
coord = coords.SkyCoord(ra=ra[0], dec=dec[0], unit=(u.deg, u.deg), frame='icrs')

In [29]:
xid = SDSS.query_region(coord, spectro='apogee', radius=2*u.arcmin)

In [30]:
sp = SDSS.get_spectra(matches=xid)

In [31]:
sp

[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x165179a90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x165155510>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1651ab590>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1651b1e90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x166306b50>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x16631b810>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x166334510>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1663490d0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x166361d50>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x166376950>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x16638b4d0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1650e8ed0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x16514ead0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1663bb890>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1663d0490>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x166

In [32]:
from astropy.io import fits

In [34]:
for i, hdu_list in enumerate(sp):
    print(f"HDUList {i}:")
    hdu_list.info()  # This prints information about each HDU in the list

    # Accessing a specific HDU (e.g., the first HDU, which typically contains the primary data)
    primary_hdu = hdu_list[0]  # Change the index as needed based on the structure of the FITS file

    # Accessing data in this HDU
    data = primary_hdu.data
    header = primary_hdu.header

    # Printing some header information (metadata)
    print(header.keys())  # Prints all header keys

HDUList 0:
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     135   ()      
  1  COADD         1 BinTableHDU     26   4621R x 8C   [E, E, E, J, J, E, E, E]   
  2  SPALL         1 BinTableHDU    488   1R x 236C   [27A, 14A, 4A, E, E, J, J, E, J, E, E, E, K, K, K, K, K, K, K, K, K, B, B, J, I, 5E, 5E, J, J, J, J, 7A, 7A, 16A, D, D, 6A, 21A, E, E, E, J, E, 24A, 10J, J, 10E, E, E, E, E, E, E, J, E, E, E, J, 5E, E, E, 10E, 10E, 10E, 5E, 5E, 5E, 5E, 5E, J, J, E, E, E, E, E, E, 16A, 9A, 12A, E, E, E, E, E, E, E, E, J, E, E, J, J, 6A, 21A, E, 35E, K, 19A, 19A, 19A, B, B, B, I, 3A, B, I, I, I, I, J, E, J, J, E, E, E, E, E, E, E, E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5J, 5J, 5J, 5E, 5J, 75E, 75E, 5E, 5E, 5E, 5J, 5E, D, D, D, D, D, D, D, D, D, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E

<generator object Header.keys at 0x165168a00>
HDUList 1:
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     135   ()      
  1  COADD         1 BinTableHDU     26   4621R x 8C   [E, E, E, J, J, E, E, E]   
  2  SPALL         1 BinTableHDU    488   1R x 236C   [27A, 14A, 4A, E, E, J, J, E, J, E, E, E, K, K, K, K, K, K, K, K, K, B, B, J, I, 5E, 5E, J, J, J, J, 7A, 7A, 16A, D, D, 6A, 21A, E, E, E, J, E, 24A, 10J, J, 10E, E, E, E, E, E, E, J, E, E, E, J, 5E, E, E, 10E, 10E, 10E, 5E, 5E, 5E, 5E, 5E, J, J, E, E, E, E, E, E, 16A, 9A, 12A, E, E, E, E, E, E, E, E, J, E, E, J, J, 6A, 21A, E, 35E, K, 19A, 19A, 19A, B, B, B, I, 3A, B, I, I, I, I, J, E, J, J, E, E, E, E, E, E, E, E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5J, 5J, 5J, 5E, 5J, 75E, 75E, 5E, 5E, 5E, 5J, 5E, D, D, D, D, D, D, D, D, D, 5E, 5E, 